# 50. データの入手・整形

In [4]:
import pandas as pd
from random import shuffle

In [11]:
'''
出来ないが？
'''
col_names = ['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP']
df = pd.read_csv("./data/NewsAggregatorDataset/newsCorpora.csv", encoding = 'UTF-8', names = col_names, sep = '\t')
extracted_data_frame = df[df['PUBLISHER'].isin(['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail'])][['TITLE', 'CATEGORY']]
row_size, _ = extracted_data_frame.shape
data_list = [{} for _ in range(row_size)]

for data_name in extracted_data_frame:
    for idx, frame in enumerate(extracted_data_frame[data_name]):
        data_list[idx][data_name] = frame


shuffle(data_list)

train_size = row_size // 10 * 8
valid_size = row_size // 10
test_size = row_size // 10

path = './work/'
file_names = ['train.txt', 'valid.txt', 'test.txt']
data_names = ['train_data', 'valid_data', 'test_data']

file_data = dict()
file_data['train_data'] = data_list[:train_size]
file_data['valid_data'] = data_list[train_size:train_size + valid_size]
file_data['test_data'] = data_list[-test_size:]


for idx, file_name in enumerate(file_names):
    with open(path + file_name, mode = 'w') as f:
        for data in file_data[data_names[idx]]:
            f.write('\t'.join([data['TITLE'], data['CATEGORY']]) + '\n')

In [12]:
data_list = list()
with open('./data/NewsAggregatorDataset/newsCorpora.csv', mode = 'r') as f:
    for line in f:
        splited_line = line.split('\t')
        if splited_line[3] in ['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']:
            data_list.append([splited_line[1], splited_line[4]])

row_size = len(data_list)
shuffle(data_list)

train_size = row_size // 10 * 8
valid_size = row_size // 10
test_size = row_size // 10

path = './work/'
file_names = ['train.txt', 'valid.txt', 'test.txt']
data_names = ['train_data', 'valid_data', 'test_data']

file_data = dict()
file_data['train_data'] = data_list[:train_size]
file_data['valid_data'] = data_list[train_size:train_size + valid_size]
file_data['test_data'] = data_list[-test_size:]


for idx, file_name in enumerate(file_names):
    with open(path + file_name, mode = 'w') as f:
        for data in file_data[data_names[idx]]:
            f.write('\t'.join([data[0], data[1]]) + '\n')

In [13]:
!wc ./work/*

    1335   15352   89234 ./work/test.txt
   10680  122223  713069 ./work/train.txt
    1335   15335   89585 ./work/valid.txt
   13350  152910  891888 total


# 51. 特徴量抽出

# 52. 学習

# 53. 予測

# 54. 正解率の計測

# 55. 混同行列の作成

# 56. 適合率，再現率，F1スコアの計測

# 57. 特徴量の重みの確認

# 58. 正則化パラメータの変更

# 59. ハイパーパラメータの探索